In [23]:
from matplotlib import pyplot as plt 
#from lines_generator import *
from hough import imshow
import numpy as np
import cv2
import keras

# Data

In [28]:
%%time

n = 50
Y = list()
patches = list()

#load all patches
for i in range(n):
    Y  += np.load('data/Y'+str(i+1)+'.npy').tolist()
    patches += np.load('data/patches'+str(i+1)+'.npy').tolist()

#add inverse
Y = [[yi,int(not yi)] for yi in Y] 

#to numpy
Y = np.array(Y).astype('uint8')
patches = np.array(patches).astype('uint16')

#add color dimension
patches = np.expand_dims(patches, axis=3)

print(patches.shape, Y.shape)

(19702, 17, 17, 1) (19702, 2)
CPU times: user 1.14 s, sys: 56.1 ms, total: 1.19 s
Wall time: 1.8 s


__x_train e x_test__

In [29]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(patches, Y, test_size=0.20, random_state=42)

In [30]:
patche_size = x_train.shape[1]

# CNN

In [27]:
from keras.layers import Conv2D, MaxPooling2D, Flatten

input_shape = (patche_size,patche_size,1)

NameError: name 'patche_size' is not defined

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train

In [8]:
model.fit(x_train, y_train,
          epochs=10,
          batch_size=128)

Epoch 1/10
15761/15761 [==============================] - 4s 252us/step - loss: 3.6192 - acc: 0.7691
Epoch 2/10
15761/15761 [==============================] - 4s 251us/step - loss: 3.6335 - acc: 0.7746
Epoch 3/10
15761/15761 [==============================] - 4s 262us/step - loss: 3.6335 - acc: 0.7746
Epoch 4/10
15761/15761 [==============================] - 4s 266us/step - loss: 3.6335 - acc: 0.7746
Epoch 5/10
15761/15761 [==============================] - 4s 249us/step - loss: 3.6335 - acc: 0.7746
Epoch 6/10
15761/15761 [==============================] - 4s 249us/step - loss: 3.6335 - acc: 0.7746
Epoch 7/10
15761/15761 [==============================] - 4s 248us/step - loss: 3.6335 - acc: 0.7746
Epoch 8/10
15761/15761 [==============================] - 4s 259us/step - loss: 3.6335 - acc: 0.7746
Epoch 9/10
15761/15761 [==============================] - 5s 291us/step - loss: 3.6335 - acc: 0.7746
Epoch 10/10
15761/15761 [==============================] - 4s 277us/step - loss: 3.6335 - a

### Evaluate

In [10]:
score = model.evaluate(x_test, y_test, batch_size=128)
score

3941/3941 [==============================] - 0s 94us/step


In [24]:
pred_test = model.predict(x_test)
pred_test[:,1].mean()

1.0

## ANN

In [ ]:
#reshape to array
size = x_train.shape
x_train = np.reshape(x_train,(size[0],size[1]*size[2]))
size = x_test.shape
x_test = np.reshape(x_test,(size[0],size[1]*size[2]))
x_train.shape,x_test.shape

In [31]:
from keras import Sequential
from keras.layers import Dense

input_dim = patche_size**2

In [12]:
# create model
model = Sequential()
model.add(Dense(1000, input_dim=input_dim, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [13]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train

In [14]:
model.fit(x_train, y_train,
          epochs=10,
          batch_size=128)

Epoch 1/10
15761/15761 [==============================] - 5s 333us/step - loss: 3.6469 - acc: 0.7717
Epoch 2/10
15761/15761 [==============================] - 4s 266us/step - loss: 3.6335 - acc: 0.7746
Epoch 3/10
15761/15761 [==============================] - 4s 269us/step - loss: 3.6335 - acc: 0.7746
Epoch 4/10
15761/15761 [==============================] - 4s 271us/step - loss: 3.6335 - acc: 0.7746
Epoch 5/10
15761/15761 [==============================] - 4s 276us/step - loss: 3.6335 - acc: 0.7746
Epoch 6/10
15761/15761 [==============================] - 4s 276us/step - loss: 3.6335 - acc: 0.7746
Epoch 7/10
15761/15761 [==============================] - 4s 278us/step - loss: 3.6335 - acc: 0.7746
Epoch 8/10
15761/15761 [==============================] - 4s 279us/step - loss: 3.6335 - acc: 0.7746
Epoch 9/10
15761/15761 [==============================] - 4s 278us/step - loss: 3.6335 - acc: 0.7746
Epoch 10/10
15761/15761 [==============================] - 5s 308us/step - loss: 3.6335 - a

### Evaluate

In [15]:
score = model.evaluate(x_test, y_test, batch_size=128)
score

3941/3941 [==============================] - 0s 74us/step


[3.6276962008158042, 0.7749302211191351]

In [16]:
pred_test = model.predict(x_test)
pred_test[:,1].mean()

1.0

## ANN Generator

In [32]:
#https://www.kaggle.com/ezietsman/simple-keras-model-with-data-generator
def dataGenerator(n_images,patche_size = 17):
    
    #open all images
    ims = list()
    masks = list()
    for i in range(n_images):
        ims.append(np.load('data/im_train'+str(i)+'.npy'))
        masks.append(np.load('data/label_train'+str(i)+'.npy'))
    ims = np.array(ims)
    masks = np.array(masks)
    
    #expand 
    size = ims.shape[1:]
    p = int((patche_size-1)/2)
    #ims = expand(ims,patche_size)
    #masks = expand(masks,patche_size)
                     
    while(1):
        #random image and pixel
        i_im = np.random.randint(n_images)
        x = np.random.randint(size[1])+p
        y = np.random.randint(size[0])+p
                     
        patche = ims[y-p:y+p+1,x-p:x+p+1]
        patche = np.expand_dims(pat, axis=2)
        label = masks[y,x]
        yield patche, [label,int(not yi)]
                     
    

In [33]:
# create model
model = Sequential()
model.add(Dense(1000, input_dim=input_dim, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [35]:
model.fit_generator(dataGenerator(5), samples_per_epoch=50, nb_epoch=10)

/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=50, epochs=10)`
  """Entry point for launching an IPython kernel.


RuntimeError: The model needs to be compiled before being used.